<a href="https://colab.research.google.com/github/MarcChen/Fewshot-ICL/blob/main/Text_To_SQL_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Text to SQL


In this lab, we'll apply what we've already learned in the previous labs to build a sqlite database using Ministral 8b. This lab is less guided than the previous ones and you'll need to refer to what you've done previously to complete each part. Moreover, this lab is more focused on prompt engineering and you have to find the best prompt and prompt strategy (system prompt? temperature value? dialog prompt style?).

For this lab, we need to use sqlite3 to execute the generated queries.
Check the doc: https://docs.python.org/3/library/sqlite3.html

<font color='red'>BE CAREFUL: you need to generate sql queries then automaticly exectute them with sqlite3 connector. DO NOT generate python code. DO NOT copy paste genereted query to the connector.</font>

<font color='green'>TIPS: sqlite3 create a file containing your db. Delete it if you need to reset the db.</font>



Lab overview:

0. Modules installation and model loading.
1. Create tables using llm.
2. Populate tables using llm.
3. Explore our tables using llm.
4. More than one table with llm.

IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...

## 0. Setup

In [ ]:
!pip install transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

from tqdm import tqdm
import pandas as pd
import sqlite3
import torch
import re

In [ ]:
# Put your hugging face token here: https://huggingface.co/docs/hub/en/security-tokens
# You need to fill the access form with your huggingface account on this link: https://huggingface.co/mistralai/Ministral-8B-Instruct-2410
hf_token = ""
llm_name = "mistralai/Ministral-8B-Instruct-2410"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left", token=hf_token)
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
    token=hf_token
)
# Set LLM on eval mode.
llm.eval()


tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSN

## 1. Create tables using llms

You need to generate and execute SQL queries to create 3 tables:
- "characters": Id (primary key), Name (str), Age (int), Profession (int).
- "characters20": same than characters.
- "skills": Id (primary key), Name (str), Profession (str).

For example, by running this code ``cursor.execute("""PRAGMA table_info(characters);""").fetchall()``.

You should have this results:

```
`[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 1, None, 0),
 (2, 'age', 'INTEGER', 1, None, 0),
 (3, 'profession', 'TEXT', 1, None, 0)]
 ```

<font color='red'>BE CAREFUL: sqlite3 doesn't have the same possibility than SQL. You may need to specify it.</font>

## 2. Populate tables using llm

You need to generate and execute SQL queries to fill in “characters” and “characters20” :
- For both, the age must be constrained between 18 and 50 (we'll assess whether the constraint is met later).
- For “characters”, generate 10 rows using the prompt. Apply the prompt 10 times (you should end up with 100 lines).
- For “characters20”, generate 20 rows using the prompt. Apply the prompt 5 times (you should also get 100 lines at the end).


For example, executing this code ``cursor.execute("SELECT * FROM characters")``.

You should get this result (with 100 rows and perhaps different values ...) :

```
[(1, 'Alice', 25, 'Artist'),
 (2, 'Bob', 35, 'Engineer'),
  ...
 (99, 'Ian', 32, 'Architect'),
 (100, 'Jane', 18, 'Dancer')]
 ```

<font color='red'> BE CAREFUL: If your generation configuration doesn't include sampling, you'll always have the same rows.</font>

<font color='green'> BONUS: In section 3, we'll compare the number of duplicated rows between the two methods. Do you have a better strategy for minimizing the number of duplicated rows? Give it a try! (create another table for this purpose) </font>

## 3. Explore our tables using llm.
  
First, you need to generate and execute SQL queries that indicate the number of duplicate rows (without ids) in each character table. To make things easier, we only ask for the number of each duplicated rows.

Here is an examples of expected results:

```
[(2,), (7,), (5,), (2,), (2,), (3,), (2,), (2,), (2,), (2,), (2,), (2,), (2,)]
```

<font color='green'> BONUS: Generate a query that returns the total count of duplicated rows. You may need to do this in several steps.</font>

Secondly, you need to generate and execute SQL queries that remove duplicate rows. To make things easier, it's not necessary to keep original duplicated lines. For example, if you have a list like this : [a, b, a, c]. We ask you to remove all the a: [b, c].

<font color='green'> BONUS: Generate a query that delete duplicated but keep the original row. [a, b, a, c] -> [a, b, c] </font>

Finaly, you need to generate and execute SQL queries that check if the age constraint is respected.

<font color='red'> BE CAREFUL: Do each step for every characters tables you have.</font>


## 4. More than one table with llm.
  
First, choose your best characters table (with the largest number of rows).


Second, generate and execute an SQL query that returns the set of unique professions in the table.


Third, generate and execute an SQL query that populates the skill tables from this set of unique professions.


Fourth, generate and execute an SQL query that verifies that the professions in the skill table exist in your characters table.


Finally, generate and execute an SQL query that returns the name of the skills associated with a character name (by profession).